Sascha Spors,
Professorship Signal Theory and Digital Signal Processing,
Institute of Communications Engineering (INT),
Faculty of Computer Science and Electrical Engineering (IEF),
University of Rostock, Germany

Tutorial Digital Signal Processing (Course #24505),
**Quantization**,
Winter Semester 2019/20

Feel free to contact lecturer frank.schultz@uni-rostock.de

- lecture: https://github.com/spatialaudio/digital-signal-processing-lecture
- tutorial: https://github.com/spatialaudio/digital-signal-processing-exercises

# Fundamentals

## Packages / Functions

We import the required packages first.

In [ ]:
# most common used packages for DSP, have a look into other scipy submodules
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import signal
#from scipy import fftpack
#from scipy import stats

Next, we should implement and validate the midtread quantizer.
We do this with two approaches. `uniform_midtread_quantizer` follows the lecture and thus directly the calculation of the quantization model. `my_quant` is a more intuitive approach using `round` rather than `floor`. 

They differentiate slightly as follows:

- `uniform_midtread_quantizer` always exhibits odd number of quantization steps
- Both quantization functions saturate $x<-1$ towards $x_q = -1$
- `uniform_midtread_quantizer` saturates $x>+1$ towards $x_q = +1$
- In the case of an odd number of quantization steps, `my_quant`  saturates $x>+1$ towards $x_q = +1$
- In the case of an even number of quantization steps, `my_quant` saturates $x>1-2^{-B}$ towards $x_q = 1-2^{-(B-1)}$.

The last case is used for AD and DA converters, since it is meaningful to code an even number of conditions with bits.
Said differently: the number range convention for analog/digital (AD) and digital/analog (DA) converters is $-1\leq x \leq 1-2^{-(B-1)}$ using $2^B$ quantization steps, $B$ denotes the number of bits, e.g. B = 16 bit for CD audio quality. Values outside this range will be saturated to their border values.

In [ ]:
def uniform_midtread_quantizer(x, Q):
    # from lecture:
    # https://github.com/spatialaudio/digital-signal-processing-lecture/blob/master/quantization/linear_uniform_characteristic.ipynb
    # commit: b00e23e
    
    # limiter
    x = np.copy(x)
    idx = np.where(np.abs(x) >= 1)
    x[idx] = np.sign(x[idx])
    # linear uniform quantization
    xQ = Q * np.floor(x/Q + 1/2)
    return xQ

def my_quant(x, N):
    # saturated midtread quantizer
    # input signal x
    # number of quantization steps N
    # if N is even: largest step is saturated to previous step, such that even
    # number of code words can be assigned and thus can be represented as bits
    tmp = N//2
    quant_steps = (np.arange(0, N) - tmp) / tmp
    xq = np.round(x*tmp) / tmp
    xq[xq < quant_steps[0]] = quant_steps[0]
    xq[xq > quant_steps[-1]] = quant_steps[-1]
    return xq

N = 8  # number of quantization steps
x = np.arange(-1.1, +1.1, 1e-3)
xq = my_quant(x, N)
xumq = uniform_midtread_quantizer(x, 1/(N//2))

plt.figure(figsize=(6, 6))
plt.plot(x, xumq, label='uniform_midtread_quantizer()')
plt.plot(x, xq, label='my_quant()')
plt.xticks(np.arange(-1, 1.25, 0.25))
plt.yticks(np.arange(-1, 1.25, 0.25))
plt.axis('equal')
plt.legend()
plt.grid(True)


# **Copyright**

The notebooks are provided as [Open Educational Resources](https://en.wikipedia.org/wiki/Open_educational_resources). Feel free to use the notebooks for your own purposes. The text is licensed under [Creative Commons Attribution 4.0](https://creativecommons.org/licenses/by/4.0/), the code of the IPython examples under the [MIT license](https://opensource.org/licenses/MIT). Please attribute the work as follows: *Frank Schultz, Digital Signal Processing - A Tutorial Featuring Computational Examples* with the URL https://github.com/spatialaudio/digital-signal-processing-exercises